<a href="https://colab.research.google.com/github/aleXyaO7/anti-Asian-BERT/blob/main/LabelPropagation_anti_asian_hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.1 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 596 kB 69.2 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
df = pd.read_excel('/content/drive/MyDrive/BERT/combined_v2.xlsx').sample(frac=1,random_state=0)
df.head()


,ID,Tweet ID,Text,label01,Source
4017,4018,1.221472e+18,RT @ratu_adil Vaccine againts deadly #WuhanCor...,0,Studies
3991,3992,1.221456e+18,RT @BumrungradH What to know about #coronaviru...,0,Studies
17865,17866,1.238900e+18,widen testing scope: all patients in govt./pvt...,0,Studies
26340,26341,1.233432e+18,I’m very here for associating this song with a...,0,DH
24079,24080,7.937718e+08,So China has bought out NBC News? Nothing like...,1,MA


In [ ]:
#convert label to integer
df = df.astype({"label01": int})
df.dtypes
df.head()

,ID,Tweet ID,Text,label01,Source
24708,24709,1.304248e+18,"Good read and great reporting, as usual, from ...",0,AL
19760,19761,1.239712e+18,@eugenegu @realdonaldtrump #chinesevirus is tr...,0,Studies
27137,27138,3.133488e+09,So @realDonaldTrump sent tons of our PPE to Ch...,0,DH
2396,2397,1.214193e+18,@coffee_ted They are more like terrorists. #Po...,0,Studies
25904,25905,1.250981e+18,Quarantine got me thinking about being a veget...,0,AY


In [ ]:
df.shape

(27290, 5)

In [ ]:
df['label01'].value_counts(normalize = True)

0    0.806303
1    0.193697
Name: label01, dtype: float64

In [ ]:
df['Text'][0]

'Are we still allowed to quote ancient Chinese proverbs, or is that racist? #RacismIsAVirus'

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
tokenized

24708    [101, 2204, 3191, 1998, 2307, 7316, 1010, 2004...
19760    [101, 1030, 8207, 12193, 1030, 2613, 5280, 190...
27137    [101, 2061, 1030, 2613, 5280, 19058, 24456, 23...
2396     [101, 1030, 4157, 1035, 6945, 2027, 2024, 2062...
25904    [101, 24209, 20486, 10196, 2288, 2033, 3241, 2...
                               ...                        
13123    [101, 19387, 1030, 17667, 22932, 2475, 1024, 2...
19648    [101, 19387, 1030, 1014, 1035, 4331, 2475, 102...
9845     [101, 1030, 20149, 8602, 2121, 2549, 1030, 152...
10799    [101, 2045, 2003, 2053, 2489, 2811, 1998, 4613...
2732     [101, 1030, 25636, 15557, 2368, 2023, 2003, 66...
Name: Text, Length: 27290, dtype: object

In [ ]:
newt = [[tokenized[i][0], tokenized[i][1]] for i in range(len(tokenized))]

In [ ]:
newt

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/BERT/covid_tweets_ZL-1.csv', delimiter='\t', header=None, names=['id', 'date', 'time', 'Text', 'label01', 'xcord', 'ycord', 'device', 'place', 'city'])

In [ ]:
df2.head()

,id,date,time,Text,label01,xcord,ycord,device,place,city
0,1309357329423372289,1.155367e+18,2020-09-25 01:01:28,I miss running around Sydney; shoppin in Hong ...,-118.411903,34.020790,Twitter for iPhone,Place,city,NaN
1,1309357478237409280,1.252062e+18,2020-09-25 01:02:04,@KazuhFPS @axzryyy @biggestlobasimp @mr_mast3r...,-82.432976,27.705530,Twitter for iPhone,Place,city,NaN
2,1309357484189065219,1.590034e+07,2020-09-25 01:02:05,Like don’t y’all know that we’re currently in ...,-80.122353,26.301197,Twitter for iPhone,Place,city,NaN
3,1309357555911520258,4.838334e+07,2020-09-25 01:02:22,Nikki is afraid of a potential Biden or even K...,-121.436218,37.723351,Twitter for iPhone,Place,city,NaN
4,1309357702506835968,1.269825e+18,2020-09-25 01:02:57,Not surprising in the slightest. Not that it a...,-73.968544,40.780708,Twitter for iPhone,Place,city,NaN


In [ ]:
df2 = df2.sample(frac=1,random_state=0)

In [ ]:
df2.head()

,id,date,time,Text,label01,xcord,ycord,device,place,city
54458,1504317777758212097,1.062513e+18,2022-03-17 00:44:41,As more COVID precautions are rolled back; it ...,-111.917023,40.765804,Twitter for iPhone,Place,city,NaN
118646,1409423262212182016,1.279343e+18,2021-06-28 04:07:46,@Quicktake She’s flatout lying. Whoever made t...,-111.780869,34.641464,Twitter for iPhone,Place,city,NaN
57401,1255975603158888455,9.682092e+17,2020-04-30 17:41:33,@ROSE6LICQUOT Is that really a question??🤔 Gas...,-79.272568,43.629311,Twitter for iPhone,Place,city,NaN
115324,1233425299267715072,1.102323e+18,2020-02-28 11:14:41,UK lookin pretty smart with Brexit atm! Corona...,-118.048897,34.168327,Twitter for iPhone,Place,city,NaN
71684,1338286085504372747,3.449322e+07,2020-12-13 19:54:01,@JocelynBenson I am so Proud to be a #Michigan...,-82.510063,27.255001,Twitter for Android,Place,city,NaN


In [ ]:
df2 = df2[['Text']]

In [ ]:
df['Text']

4017     RT @ratu_adil Vaccine againts deadly #WuhanCor...
3991     RT @BumrungradH What to know about #coronaviru...
17865    widen testing scope: all patients in govt./pvt...
26340    I’m very here for associating this song with a...
24079    So China has bought out NBC News? Nothing like...
                               ...                        
4593     #CoronavirusOutbreak | Indian Embassy in Beiji...
22013    obama 'team' failed bigly during 2009 swine h1...
1044     he's trying to say *wazu* do wazu fives times ...
26298    I am so here for @KeithUrban singing @SteveWin...
9711     @JoeB14ck ccp is the biggest spreader #WuhanVi...
Name: Text, Length: 27290, dtype: object

In [ ]:
df2['Text']

54458     As more COVID precautions are rolled back; it ...
118646    @Quicktake She’s flatout lying. Whoever made t...
57401     @ROSE6LICQUOT Is that really a question??🤔 Gas...
115324    UK lookin pretty smart with Brexit atm! Corona...
71684     @JocelynBenson I am so Proud to be a #Michigan...
                                ...                        
176963    @girlsreallyrule @eturleye @NYGovCuomo Super e...
117952    This epidemic; this pandemic; this time of our...
173685    I still remember my family gc always saying “o...
43567     Summary of Gov. Reynold’s press conference as ...
199340    Treating myself to dinner 🍽 🇫🇷 🏡 photos doesn’...
Name: Text, Length: 200001, dtype: object

In [ ]:
df['Text'][0]

'Are we still allowed to quote ancient Chinese proverbs, or is that racist? #RacismIsAVirus'

In [ ]:
df2['Text'][0]

'I miss running around Sydney; shoppin in Hong Kong; cuddling with lion cubs in South Africa; and taking pictures with folks ALL DAY in Tokyo. I miss crawling out the club at 9am in South Korea and photo Shoots in Santorini; and hosting events in Dubai. I miss my life. FU Covid https://t.co/EJM3mcR5kY'

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized2 = df2['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

ValueError: ignored

In [ ]:
from sklearn.semi_supervised import LabelPropagation

lp = LabelPropagation()
lp.fit(newt, df['label01'])

LabelPropagation()

In [ ]:
lp.predict(df2)

ValueError: ignored